# Proyecto Final Algebra Lineal 2017
---

El proyecto del curso consiste en la implementación del [Algortimo de Canny](https://es.wikipedia.org/wiki/Algoritmo_de_Canny) para la detección de bordes en imágenes y videos. Para ello, se le entrega al estudiante toda la implementación del algoritmo en [OpenCV](http://opencv.org/) y su objetivo es **implementar las funciones de OpenCV con matrices**. 

Las funciones a modificar serán las siguientes:
```python
cv2.GaussianBlur()
cv2.Sobel()
```

No es necesario que modifique más del código que se le solicita, pero puede hacer las modificaciones pertinentes si ese fuera el caso. Dentro del código se le indica en qué parte debe de ser modificado, por ejemplo:

```python
'''Modificar esta función'''
    sobel = cv2.Sobel(img, cv2.CV_64F, x, y, ksize=sobel_kernel)
``` 

Se recomienda buscar maneras de optimizar los cálculos de sus filtros, pues más adelante existe una aplicación en tiempo real donde es crucial que sus cálculos sean lo más rápidos posibles. Las dimensiones de los *kernels* no pueden ser modificadas.

## 0. Conceptos
Para poder modificar las funciones solicitadas es necesario que comprenda algunos conceptos, por lo cual es necesario que defina:

1. ¿En que consiste la convolución y cómo puede ser calculada? (Puede dar una definición matemática y/o ejemplos).

2. ¿Qué es el *kernel* de un filtro y para que se utiliza?

3. ¿En qué consiste un filtro Gaussiano?

4. ¿En qué consiste el Filtro Sobel? ¿para qué sirve?

3. Describa el algoritmo de detección de bordes Canny.

In [36]:
import numpy as np
import matplotlib.pylab as plt
import cv2
from time import time
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import math

%matplotlib inline

In [37]:
#Kernels para Sobel
kernel3SobelY = [(-1,-2,-1), (0,0,0), (1,2,1)]
kernel3SobelX = [(-1,0,1), (-2,0,2), (-1,0,1)]

kernel5SobelX = [(1,2,0,-2,-1), (4,8,0,-8-4), (6,12,0,-12,-6), (4,8,0,-8,-4), (1,2,0,-2,-1)]
kernel5SobelY = [(-1,-4,-6,-4,-1), (-2,-8,-12,-8,-2), (0,0,0,0,0), (2,8,12,8,2), (1,4,6,4,1)]

#Kernels para Gaussian
Kernel3Gaussian = [(0,0,0), (0,1,0), (0,0,0)]
Kernel5Gaussian = [(1,2,3,2,1), (2,7,11,7,2), (3,11,17,11,3), (2,7,11,7,2), (1,2,3,2,1)]

#Filtro Gaussian
def Convolucion3(ImagenGrises):
    tiempoI = time()
    Gausseano = np.zeros((len(ImagenGrises), int(ImagenGrises.size/len(ImagenGrises))))
    for i in range (1, len(ImagenGrises) - 1):
        for j in range (1, int(ImagenGrises.size/len(ImagenGrises)) - 1):
            Gausseano[i, j] = int((ImagenGrises[i - 1][j - 1] * Kernel3Gaussian[0][0]) + (ImagenGrises[i - 1][j] * Kernel3Gaussian[0][1]) + (ImagenGrises[i - 1][j + 1] * Kernel3Gaussian[0][2]) + (ImagenGrises[i][j - 1] * Kernel3Gaussian[1][0]) + (ImagenGrises[i][j] * Kernel3Gaussian[1][1]) + (ImagenGrises[i][j + 1] * Kernel3Gaussian[1][2]) + (ImagenGrises[i + 1][j - 1] * Kernel3Gaussian[2][0]) + (ImagenGrises[i + 1][j] * Kernel3Gaussian[2][1]) + (ImagenGrises[i + 1][j + 1] * Kernel3Gaussian[2][2]))
    tiempoF = time()
    print("Tiempo para Gaussian (3x3): " + str(tiempoF - tiempoI))
    return Gausseano

def Convolucion5(ImagenGrises):
    tiempoI = time()
    Gausseano = np.zeros((len(ImagenGrises), int(ImagenGrises.size/len(ImagenGrises))))
    for i in range (2, len(ImagenGrises) - 2):
        for j in range(2, int(ImagenGrises.size/len(ImagenGrises)) - 2):
            Gausseano[i][j] = int((ImagenGrises[i - 2][j - 2] * Kernel5Gaussian[0][0]) + (ImagenGrises[i - 2][j - 1] * Kernel5Gaussian[0][1]) + (ImagenGrises[i - 2][j] * Kernel5Gaussian[0][2]) + (ImagenGrises[i - 2][j + 1] * Kernel5Gaussian[0][3]) + (ImagenGrises[i - 2][j + 2] * Kernel5Gaussian[0][4])) + int((ImagenGrises[i - 1][j - 2] * Kernel5Gaussian[1][0]) + (ImagenGrises[i - 1][j - 1] * Kernel5Gaussian[1][1]) + (ImagenGrises[i - 1][j] * Kernel5Gaussian[1][2]) + (ImagenGrises[i - 1][j + 1] * Kernel5Gaussian[1][3]) + (ImagenGrises[i - 1][j + 2] * Kernel5Gaussian[1][4])) + int((ImagenGrises[i][j - 2] * Kernel5Gaussian[2][0]) + (ImagenGrises[i][j - 1] * Kernel5Gaussian[2][1]) + (ImagenGrises[i][j] * Kernel5Gaussian[2][2]) + (ImagenGrises[i][j + 1] * Kernel5Gaussian[2][3]) + (ImagenGrises[i][j + 2] * Kernel5Gaussian[2][4])) + int((ImagenGrises[i + 1][j - 2] * Kernel5Gaussian[3][0]) + (ImagenGrises[i + 1][j - 1] * Kernel5Gaussian[3][1]) + (ImagenGrises[i + 1][j] * Kernel5Gaussian[3][2]) + (ImagenGrises[i + 1][j + 1] * Kernel5Gaussian[3][3]) + (ImagenGrises[i + 1][j + 2] * Kernel5Gaussian[3][4])) + int((ImagenGrises[i + 2][j - 2] * Kernel5Gaussian[4][0]) + (ImagenGrises[i + 2][j - 1] * Kernel5Gaussian[4][1]) + (ImagenGrises[i + 2][j] * Kernel5Gaussian[4][2]) + (ImagenGrises[i + 2][j + 1] * Kernel5Gaussian[4][3]) + (ImagenGrises[i + 2][j + 2] * Kernel5Gaussian[4][4]))
    tiempoF = time()
    print("Tiempo para Gaussian (5x5): " + str(tiempoF - tiempoI))
    return Gausseano

def Sobel3X(GaussianBlur):
    tiempoI = time()
    Sobel = np.zeros((len(GaussianBlur), int(GaussianBlur.size/len(GaussianBlur))))
    for i in range (1, len(GaussianBlur) - 1):
        for j in range (1, int(GaussianBlur.size/len(GaussianBlur)) - 1):
            Sobel[i][j] = int((GaussianBlur[i - 1][j - 1] * kernel3SobelX[0][0]) + (GaussianBlur[i - 1][j] * kernel3SobelX[0][1]) + (GaussianBlur[i - 1][j + 1] * kernel3SobelX[0][2]) + (GaussianBlur[i][j - 1] * kernel3SobelX[1][0]) + (GaussianBlur[i][j] * kernel3SobelX[1][1]) + (GaussianBlur[i][j + 1] * kernel3SobelX[1][2]) + (GaussianBlur[i + 1][j - 1] * kernel3SobelX[2][0]) + (GaussianBlur[i + 1][j] * kernel3SobelX[2][1]) + (GaussianBlur[i + 1][j + 1] * kernel3SobelX[2][2]))
    tiempoF = time()
    print("Tiempo para Sobel en X (3x3): " + str(tiempoF - tiempoI))
    return Sobel
    
def Sobel3Y(GaussianBlur):
    tiempoI = time()
    Sobel = np.zeros((len(GaussianBlur), int(GaussianBlur.size/len(GaussianBlur))))
    for j in range (1, int(GaussianBlur.size/len(GaussianBlur)) - 1):
        for i in range (1, len(GaussianBlur) - 1):
            Sobel[i][j] = int((GaussianBlur[i - 1][j - 1] * kernel3SobelY[0][0]) + (GaussianBlur[i - 1][j] * kernel3SobelY[0][1]) + (GaussianBlur[i - 1][j + 1] * kernel3SobelY[0][2]) + (GaussianBlur[i][j - 1] * kernel3SobelY[1][0]) + (GaussianBlur[i][j] * kernel3SobelY[1][1]) + (GaussianBlur[i][j + 1] * kernel3SobelY[1][2]) + (GaussianBlur[i + 1][j - 1] * kernel3SobelY[2][0]) + (GaussianBlur[i + 1][j] * kernel3SobelY[2][1]) + (GaussianBlur[i + 1][j + 1] * kernel3SobelY[2][2]))
    tiempoF = time()
    print("Tiempo para Sobel en Y (3x3): " + str(tiempoF - tiempoI))
    return Sobel
    
def Sobel5X(GaussianBlur):
    tiempoI = time()
    Sobel = np.zeros((len(GaussianBlur), int(GaussianBlur.size/len(GaussianBlur))))
    print(GaussianBlur.shape)
    print(Sobel.shape)
    for i in range (2, len(GaussianBlur) - 2):
        for j in range(2, int(GaussianBlur.size/len(GaussianBlur)) - 2):
            Sobel[i][j] = int((GaussianBlur[i - 2][j - 2] * kernel5SobelX[0][0]) + (GaussianBlur[i - 2][j - 1] * kernel5SobelX[0][1]) + (GaussianBlur[i - 2][j] * kernel5SobelX[0][2]) + (GaussianBlur[i - 2][j + 1] * kernel5SobelX[0][3]) + (GaussianBlur[i - 2][j + 2] * kernel5SobelX[0][4])) + int((GaussianBlur[i - 1][j - 2] * kernel5SobelX[1][0]) + (GaussianBlur[i - 1][j - 1] * kernel5SobelX[1][1]) + (GaussianBlur[i - 1][j] * kernel5SobelX[1][2]) + (GaussianBlur[i - 1][j + 1] * kernel5SobelX[1][3]) + (GaussianBlur[i - 1][j + 2] * kernel5SobelX[1][4])) + int((GaussianBlur[i][j - 2] * kernel5SobelX[2][0]) + (GaussianBlur[i][j - 1] * kernel5SobelX[2][1]) + (GaussianBlur[i][j] * kernel5SobelX[2][2]) + (GaussianBlur[i][j + 1] * kernel5SobelX[2][3]) + (GaussianBlur[i][j + 2] * kernel5SobelX[2][4])) + int((GaussianBlur[i + 1][j - 2] * kernel5SobelX[3][0]) + (GaussianBlur[i + 1][j - 1] * kernel5SobelX[3][1]) + (GaussianBlur[i + 1][j] * kernel5SobelX[3][2]) + (GaussianBlur[i + 1][j + 1] * kernel5SobelX[3][3]) + (GaussianBlur[i + 1][j + 2] * kernel5SobelX[3][4])) + int((GaussianBlur[i + 2][j - 2] * kernel5SobelX[4][0]) + (GaussianBlur[i + 2][j - 1] * kernel5SobelX[4][1]) + (GaussianBlur[i + 2][j] * kernel5SobelX[4][2]) + (GaussianBlur[i + 2][j + 1] * kernel5SobelX[4][3]) + (GaussianBlur[i + 2][j + 2] * kernel5SobelX[4][4]))
    tiempoF = time()
    print("Tiempo para Sobel en X (5x5): " + str(tiempoF - tiempoI))
    return Sobel
    
def Sobel5Y(GaussianBlur):
    tiempoI = time()
    Sobel = np.zeros((len(GaussianBlur), int(GaussianBlur.size/len(GaussianBlur))))
    for j in range(2, int(GaussianBlur.size/len(GaussianBlur)) - 2):
        for i in range(2, len(GaussianBlur) - 2):
            Sobel[i][j] = int((GaussianBlur[i - 2][j - 2] * kernel5SobelY[0][0]) + (GaussianBlur[i - 2][j - 1] * kernel5SobelY[0][1]) + (GaussianBlur[i - 2][j] * kernel5SobelY[0][2]) + (GaussianBlur[i - 2][j + 1] * kernel5SobelY[0][3]) + (GaussianBlur[i - 2][j + 2] * kernel5SobelY[0][4])) + int((GaussianBlur[i - 1][j - 2] * kernel5SobelY[1][0]) + (GaussianBlur[i - 1][j - 1] * kernel5SobelY[1][1]) + (GaussianBlur[i - 1][j] * kernel5SobelY[1][2]) + (GaussianBlur[i - 1][j + 1] * kernel5SobelY[1][3]) + (GaussianBlur[i - 1][j + 2] * kernel5SobelY[1][4])) + int((GaussianBlur[i][j - 2] * kernel5SobelY[2][0]) + (GaussianBlur[i][j - 1] * kernel5SobelY[2][1]) + (GaussianBlur[i][j] * kernel5SobelY[2][2]) + (GaussianBlur[i][j + 1] * kernel5SobelY[2][3]) + (GaussianBlur[i][j + 2] * kernel5SobelY[2][4])) + int((GaussianBlur[i + 1][j - 2] * kernel5SobelY[3][0]) + (GaussianBlur[i + 1][j - 1] * kernel5SobelY[3][1]) + (GaussianBlur[i + 1][j] * kernel5SobelY[3][2]) + (GaussianBlur[i + 1][j + 1] * kernel5SobelY[3][3]) + (GaussianBlur[i + 1][j + 2] * kernel5SobelY[3][4])) + int((GaussianBlur[i + 2][j - 2] * kernel5SobelY[4][0]) + (GaussianBlur[i + 2][j - 1] * kernel5SobelY[4][1]) + (GaussianBlur[i + 2][j] * kernel5SobelY[4][2]) + (GaussianBlur[i + 2][j + 1] * kernel5SobelY[4][3]) + (GaussianBlur[i + 2][j + 2] * kernel5SobelY[4][4]))
    tiempoF = time()
    print("Tiempo para Sobel en Y (5x5): " + str(tiempoF - tiempoI))
    return Sobel

def Sobel5(GaussianBlur):
    x = Sobel5X(GaussianBlur)
    y = Sobel5Y(GaussianBlur)
    Sobel = np.zeros((len(GaussianBlur), int(GaussianBlur.size/len(GaussianBlur))))
    for i in range (0,len(GaussianBlur) - 1):
        for j in range(0,int(GaussianBlur.size/len(GaussianBlur)) - 1):
            Sobel[i][j] = int(math.sqrt((x[i][j]**2)+ (y[i][j]**2)))
            if Sobel[i][j] > 255:
                Sobel[i][j] = 255
    return Sobel

def Sobel3(GaussianBlur):
    x = Sobel3X(GaussianBlur)
    y = Sobel3Y(GaussianBlur)
    Sobel = np.zeros((len(GaussianBlur), int(GaussianBlur.size/len(GaussianBlur))))
    for i in range (0,len(GaussianBlur) - 1):
        for j in range(0,int(GaussianBlur.size/len(GaussianBlur)) - 1):
            Sobel[i][j] = int(math.sqrt((x[i][j]**2)+ (y[i][j]**2)))
            if Sobel[i][j] > 255:
                Sobel[i][j] = 255
    return Sobel

In [38]:
def gaussian_blur(img, kernel = 5):
    '''Modificar esta función'''
    #return cv2.GaussianBlur(img,(kernel,kernel),0)
    if kernel == 5:
        return Convolucion5(img)
    else:
        return Convolucion3(img)

def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Calculate directional gradient
    if orient == 'x':
        x,y = 1,0
    elif orient == 'y':
        x,y = 0,1
        
    '''Modificar esta función'''
    #sobel = cv2.Sobel(img, cv2.CV_64F, x, y, ksize=sobel_kernel)
    if sobel_kernel==3:
        if orient=="x":
            sobel = Sobel3X(img)
        else:
            sobel = Sobel3Y(img)
    else:
        if orient=="x":
            sobel = Sobel3X(img)
        else:
            sobel = Sobel3Y(img)
    
    # Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobel)

    # Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    
    # Create a mask of 1's where the scaled gradient magnitude 
    # is > thresh_min and < thresh_max
    sbinary = np.zeros_like(scaled_sobel)
    sbinary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return sbinary

def mag_thresh(image, sobel_kernel=3, mag_thresh=(0, 255)):
    # Calculate gradient magnitude
    # Take the gradient in x and y separately
    '''Modificar esta función'''
    #sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    if sobel_kernel == 3:
        sobelx = Sobel3X(image)
    else:
        sobelx = Sobel5X(image)
    '''Modificar esta función'''
    #sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    if sobel_kernel == 3:
        sobely = Sobel3Y(image)
    else:
        sobely = Sobel5Y(image)
    
    # Calculate the magnitude 
    abs_sobel = np.sqrt(sobelx**2 +sobely**2)

    # Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    
    # Create a binary mask where mag thresholds are met
    sbinary = np.zeros_like(scaled_sobel)
    sbinary[(scaled_sobel >= mag_thresh[0]) & (scaled_sobel <= mag_thresh[1])] = 1
    return sbinary

def dir_threshold(image, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Calculate gradient direction
    # Take the gradient in x and y separately
    '''Modificar esta función'''
    #sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    if sobel_kernel == 3:
        sobelx = Sobel3X(image)
    else:
        sobelx = Sobel5X(image)
    '''Modificar esta función'''
    #sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    if sobel_kernel == 3:
        sobely = Sobel3Y(image)
    else:
        sobely = Sobel5Y(image)

    # Take the absolute value of the x and y gradients
    abs_sobelx = np.absolute(sobelx)   
    abs_sobely = np.absolute(sobely)
    
    # Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    grad_dir = np.arctan2(abs_sobely, abs_sobelx)
    
    # Create a binary mask where direction thresholds are met
    sbinary = np.zeros_like(grad_dir)
    sbinary[(grad_dir >= thresh[0]) & (grad_dir <= thresh[1])] = 1
    return sbinary
print("corre")

corre


## 1. Prueba en Imagen

In [39]:
# Test filter on single image
im = plt.imread("elise.jpg")
gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)

# Gaussian Blur Filtering
gray = gaussian_blur(gray, kernel = 3)

# Choose a Sobel kernel size
ksize = 3 # Choose a larger odd number to smooth gradient measurements

# Apply each of the thresholding functions
gradx = abs_sobel_thresh(gray, orient='x', sobel_kernel=ksize, thresh=(50, 100))
grady = abs_sobel_thresh(gray, orient='y', sobel_kernel=ksize, thresh=(50, 100))
mag_binary = mag_thresh(gray, sobel_kernel=ksize, mag_thresh=(20, 100))
dir_binary = dir_threshold(gray, sobel_kernel=ksize, thresh=(0, np.pi/2))

# Create combined image with sobel filters applied
combined = np.zeros_like(dir_binary)
combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1

rgb_combined = np.asarray(np.dstack((combined, combined, combined)), dtype=np.float64)

Tiempo para Gaussian (5x5): 22.980477571487427
Tiempo para Sobel en X (3x3): 2.488116502761841
Tiempo para Sobel en Y (3x3): 2.3653154373168945
(480, 854)
(480, 854)


IndexError: tuple index out of range

In [ ]:
# Plot the result
f, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2, 3, figsize=(10, 15))
f.tight_layout()
ax1.imshow(im)
ax1.set_title('Original Image', fontsize=20)
ax2.imshow(gray, cmap='gray')
ax2.set_title('Gray Image', fontsize=20)
ax3.imshow(rgb_combined, cmap='gray')
ax3.set_title('Combined Sobel Image', fontsize=20)
ax4.imshow(gradx, cmap='gray')
ax4.set_title('GradX Image', fontsize=20)
ax5.imshow(grady, cmap='gray')
ax5.set_title('GradY Image', fontsize=20)
ax6.imshow(((mag_binary == 1) & (dir_binary == 1)), cmap='gray')
ax6.set_title('Thresholded Grad. Dir.', fontsize=20)

plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
plt.show() 

## 2. Prueba en Videos

In [ ]:
def process_image(image):
    
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gray = gaussian_blur(gray, kernel = 5)
    # Choose a Sobel kernel size
    ksize = 3 # Choose a larger odd number to smooth gradient measurements

    # Apply each of the thresholding functions
    gradx = abs_sobel_thresh(gray, orient='x', sobel_kernel=ksize, thresh=(50, 100))
    grady = abs_sobel_thresh(gray, orient='y', sobel_kernel=ksize, thresh=(50, 100))
    mag_binary = mag_thresh(gray, sobel_kernel=ksize, mag_thresh=(20, 100))
    dir_binary = dir_threshold(gray, sobel_kernel=ksize, thresh=(0, np.pi/4))
    
    combined = np.zeros_like(dir_binary)
    combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    
    
    rgb_combined = np.asarray(np.dstack((combined, combined, combined)), dtype=np.uint8)
    
    retval, threshold = cv2.threshold(rgb_combined, 0, 255, cv2.THRESH_BINARY)
    
    return threshold

In [ ]:
filter_output = 'Batman_Filter.mp4'
clip1 = VideoFileClip("Batman.mp4")
batman_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time batman_clip.write_videofile(filter_output, audio=False)

In [ ]:
HTML("""
<video width="720" height="480" controls>
  <source src="{0}">
</video>
""".format(filter_output))

In [ ]:
cap = cv2.VideoCapture(0)  # 0 for Python 3, 1 for Python 2. No idea why?!!

out = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*"MJPG"), 20,(640,480))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        frame = cv2.flip(frame,180)
        new_frame = process_image(frame)
        out.write(new_frame)

        cv2.imshow('frame',new_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()